In [1]:
from blimp.preprocessing.nd2_to_ome_tiff import _nd2_to_ome_tiff

13-Dec-22 08:33:48 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [9]:
from nd2reader import ND2Reader
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from aicsimageio.types import PhysicalPixelSizes

In [7]:
from nd2reader.raw_metadata import RawMetadata

In [51]:
acquisition_order = 'CZT'
in_file_path = '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_144534_824/WellE05_ChannelGFP,AF647_Seq0001.nd2'
images = ND2Reader(in_file_path)
metadata_dict = images.parser._raw_metadata.__dict__
start_time = metadata_dict['date']
acquisition_times = [t for t in images.parser._raw_metadata.acquisition_times]

In [117]:
import pandas as pd
df = pd.DataFrame(
    data={
        'stage_x' : images.parser._raw_metadata.x_data,
        'stage_y' : images.parser._raw_metadata.y_data,
        'stage_z' : images.parser._raw_metadata.z_data,
        'acquisition_time' : acquisition_times,
        'stage_z_id' : list(metadata_dict['z_levels'])*(images.sizes['t']*images.sizes['v']),
        'field_id' : list(np.repeat(range(1,1+images.sizes['v']),images.sizes['z']))*images.sizes['t']})
df['absolute_time']=[start_time + datetime.timedelta(seconds=x) for x in df['acquisition_time']]

In [118]:
df

,stage_x,stage_y,stage_z,acquisition_time,stage_z_id,field_id,absolute_time
0,12347.1,3281.8,6107.96,130.323239,0,1,2022-10-10 15:00:30.323239
1,12347.1,3281.8,6108.96,131.151798,1,1,2022-10-10 15:00:31.151798
2,12347.2,3281.8,6109.96,132.002983,2,1,2022-10-10 15:00:32.002983
3,12347.2,3281.8,6110.96,132.786543,3,1,2022-10-10 15:00:32.786543
4,12347.1,3281.8,6111.96,133.636591,4,1,2022-10-10 15:00:33.636591
...,...,...,...,...,...,...,...
427,12358.6,4599.5,6108.34,762.052066,4,16,2022-10-10 15:11:02.052066
428,12358.6,4599.4,6109.34,762.892270,5,16,2022-10-10 15:11:02.892270
429,12358.6,4599.4,6110.34,763.675718,6,16,2022-10-10 15:11:03.675718
430,12358.6,4599.4,6111.34,764.437425,7,16,2022-10-10 15:11:04.437425


In [79]:
list(metadata_dict['z_levels'])*(images.sizes['t']*images.sizes['v'])

432

In [99]:
images.parser._raw_metadata.image_text_info

{b'SLxImageTextInfo': {b'TextInfoItem_0': b'',
  b'TextInfoItem_1': b'',
  b'TextInfoItem_2': b'',
  b'TextInfoItem_3': b'',
  b'TextInfoItem_4': b'',
  b'TextInfoItem_5': b'Metadata:\r\nDimensions: T(3) x XY(16) x Z(9) x \xce\xbb(1)\r\nCamera Name: AX\r\nNumerical Aperture: 0.75\r\nRefractive Index: 1\r\nNumber of Picture Planes: 2\r\nPlane #1:\r\n Name: GFP\r\n Component Count: 1\r\n Modality: Widefield Fluorescence, Laser Scan Confocal\r\n Camera Settings:   Configuration:\r\n  \tDetector mode: Multi-Channel\r\n  \tScanner: Resonant Bidirectional\r\n  \tScan Mode: Band\r\n  \tLine Averaging: 4x\r\n  \tResolution: 1024 x 1024\r\n  \tChannel Series: Minimal Crosstalk\r\n  \tSeries Mode: Line Series\r\n  \t\tChannel Series Pass 1:\r\n  \t\t\t[AF647]\r\n  \t\tChannel Series Pass 2:\r\n  \t\t\t[GFP]\r\n  \r\n  Components:\r\n  Component GFP:\r\n  \tDetector: 2\r\n  \tEmission Range: 499 nm - 551 nm\r\n  \tGain: 30.0\r\n  \tOffset: 0\r\n  \tLaser 488 nm:\r\n  \t\tEnabled\r\n  \t\tPower: 6

In [11]:
in_file_path = '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_144534_824/WellE05_ChannelGFP,AF647_Seq0000.nd2'
images = ND2Reader(in_file_path)
for x,y in images.parser._raw_metadata.image_text_info.items():
        for i,j in y.items():
            print(f"{i.decode()} => {j.decode()}")

TextInfoItem_0 => 
TextInfoItem_1 => 
TextInfoItem_2 => 
TextInfoItem_3 => 
TextInfoItem_4 => 
TextInfoItem_5 => Metadata:
Dimensions: T(3) x XY(16) x Z(9) x λ(1)
Camera Name: AX
Numerical Aperture: 0.75
Refractive Index: 1
Number of Picture Planes: 2
Plane #1:
 Name: GFP
 Component Count: 1
 Modality: Widefield Fluorescence, Laser Scan Confocal
 Camera Settings:   Configuration:
  	Detector mode: Multi-Channel
  	Scanner: Resonant Bidirectional
  	Scan Mode: Band
  	Line Averaging: 4x
  	Resolution: 1024 x 1024
  	Channel Series: Minimal Crosstalk
  	Series Mode: Line Series
  		Channel Series Pass 1:
  			[AF647]
  		Channel Series Pass 2:
  			[GFP]
  
  Components:
  Component GFP:
  	Detector: 2
  	Emission Range: 499 nm - 551 nm
  	Gain: 30.0
  	Offset: 0
  	Laser 488 nm:
  		Enabled
  		Power: 6.0
  
  Component AF647:
  	Detector: 4
  	Emission Range: 653 nm - 726 nm
  	Gain: 25.0
  	Offset: 0
  	Laser 640 nm:
  		Enabled
  		Power: 1.8
  
  Scan Area:
  	Zoom: 2.00
  	Offset Y

In [ ]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_144534_824/WellE05_ChannelGFP,AF647_Seq0000.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_144534_824/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_144534_824/OME-TIFF-MIP')

In [ ]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/WellE06_ChannelGFP,AF647_Seq0000.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF-MIP')

In [ ]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/WellE06_ChannelGFP,AF647_Seq0001.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF-MIP')

In [2]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/WellE06_ChannelGFP_Seq0002.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF-MIP')

ValueError: axes ['c'] do not exist

In [3]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/WellE06_ChannelGFP,AF647_Seq0003.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF-MIP')

/srv/scratch/z3532965/anaconda3/envs/berrylab-default/lib/python3.8/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


PosixPath('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF/WellE06_ChannelGFP,AF647_Seq0003_0016.ome.tiff')

In [4]:
_nd2_to_ome_tiff('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/WellE06_ChannelGFP,AF647_Seq0004.nd2',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF',
                '/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF-MIP')

PosixPath('/srv/scratch/berrylab/z3532965/Nikon_AX_QPI/20221010_BleachChase_POLR2A/20221010_160654_024/OME-TIFF/WellE06_ChannelGFP,AF647_Seq0004_0016.ome.tiff')